# Получение текстов отзывов по списку отелей (со ссылками), скачанному ранее

### В отличие от названия, ссылка еще служит более надежным идентификатором отеля

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys
import numpy as np
from tqdm import tqdm
from fake_useragent import UserAgent

from IPython.display import clear_output

In [2]:
import matplotlib.pyplot as plt

## Интересующие данные: оценка, дата, текст, автор, время с момента посещения

In [3]:
root_link='https://tophotels.ru'

In [4]:
#main_link="https://tophotels.ru/hotels/1/20113"

In [5]:
#if len(sys.argv)>1:
    #fname=sys.argv[1]
#else:
    #fname="Crimea_hotel_stats_clean.csv"

In [6]:
fname="Crimea_hotel_stats_clean.csv" # running in Jupyter

In [7]:
hotel_data=pd.read_csv(fname)

In [27]:
hotel_data.head(10)

,title,area,rating,n_reviews,reference
0,MRIYA RESORT & SPA (МРИЯ РЕЗОРТ энд СПА) 5*,"Россия, Крым, Ялта",4.79,589,https://tophotels.ru/hotel/al206891/reviews
1,Ялта-Интурист 4*,"Россия, Крым, Ялта",3.90,756,https://tophotels.ru/hotel/al19733/reviews
2,Азовский,"Россия, Крым, Керчь",4.41,399,https://tophotels.ru/hotel/al142744/reviews
3,Porto Mare 4*,"Россия, Крым, Алушта",4.52,260,https://tophotels.ru/hotel/al19743/reviews
4,Нева,"Россия, Крым, Алушта, Алушта-центр",4.51,245,https://tophotels.ru/hotel/al35508/reviews
5,Империя,"Россия, Крым, Евпатория, Евпатория-центр",4.73,216,https://tophotels.ru/hotel/al62766/reviews
6,Aquamarine Resort & SPA 5*,"Россия, Крым, Севастополь, Севастополь-центр",4.72,218,https://tophotels.ru/hotel/al64527/reviews
7,Парк-отель Марат,"Россия, Крым, Гаспра",3.57,266,https://tophotels.ru/hotel/al35596/reviews
8,Таврия,"Россия, Крым, Евпатория, Евпатория-центр",4.34,211,https://tophotels.ru/hotel/al36556/reviews
9,GOLDEN RESORT (Голден Резорт) 3*,"Россия, Крым, Алушта",4.61,199,https://tophotels.ru/hotel/al19774/reviews


**Крым очень большой** (а выкачивание отзывов совпало с пиком туристического сезона), **поэтому для анализа отзывов ограничимся Южным Берегом Крыма (ЮБК), в который включим Ялту, Алушту и Алупку**.

(При этом общая статистика без текстов есть и для всего Крыма)

In [70]:
def UBK(x):
    return "Алушта" in x or "Ялта" in x or "Алупка" in x
def Sebastopol(x):
    return "Севастополь" in x
def Evpatoriya(x):
    return "Евпатория" in x
def Sudak(x):
    return "Судак" in x

In [71]:
UBK=hotel_data.iloc[3:,:].loc[hotel_data.area.apply(lambda x: UBK(x)),:]

In [73]:
UBK.head()

,title,area,rating,n_reviews,reference
3,Porto Mare 4*,"Россия, Крым, Алушта",4.52,260,https://tophotels.ru/hotel/al19743/reviews
4,Нева,"Россия, Крым, Алушта, Алушта-центр",4.51,245,https://tophotels.ru/hotel/al35508/reviews
9,GOLDEN RESORT (Голден Резорт) 3*,"Россия, Крым, Алушта",4.61,199,https://tophotels.ru/hotel/al19774/reviews
10,Ялта-Круглый Год 3*,"Россия, Крым, Ялта, Ялта-центр",4.58,175,https://tophotels.ru/hotel/al102106/reviews
18,Санаторий Кирова,"Россия, Крым, Ялта, Ялта-центр",3.94,129,https://tophotels.ru/hotel/al19754/reviews


In [60]:
Seb=hotel_data.iloc[3:,:].loc[hotel_data.area.apply(lambda x: Sebastopol(x)),:]
Evp=hotel_data.iloc[3:,:].loc[hotel_data.area.apply(lambda x: Evpatoriya(x)),:]
Sud=hotel_data.iloc[3:,:].loc[hotel_data.area.apply(lambda x: Sudak(x)),:]

### Немного статистики по районам Крыма:

In [72]:
print('ЮБК:\n', 'Hotels: ', len(UBK), '\n', 'Reviews: ', UBK.n_reviews.sum(), '\n')
print('Севастополь:\n', 'Hotels: ', len(Seb), '\n', 'Reviews: ', Seb.n_reviews.sum(), '\n')
print('Евпатория:\n', 'Hotels: ', len(Evp), '\n', 'Reviews: ', Evp.n_reviews.sum(), '\n')
print('Судак:\n', 'Hotels: ', len(Sud), '\n', 'Reviews: ', Sud.n_reviews.sum(), '\n')

ЮБК:
 Hotels:  322 
 Reviews:  4252 

Севастополь:
 Hotels:  121 
 Reviews:  960 

Евпатория:
 Hotels:  151 
 Reviews:  2003 

Судак:
 Hotels:  96 
 Reviews:  603 



In [11]:
len(hotel_data)

1239

In [10]:
hotel_data.n_reviews.sum()

14650

In [62]:
def page_link(page, link):
    return link+'?page='+str(page)

In [63]:
def get_reviews(link): # отзывы по ссылке, относящейся к конкретному отелю
    response = requests.get(link, headers={'User-Agent': UserAgent().chrome})
    soup = BeautifulSoup(response.content, "html.parser")
    all_rev=soup.find("div",attrs={'class':'reviews-list'})
    reviews=all_rev.findAll("div",attrs={'class':'tape-rev__item'}) # !!!
    return None if not reviews else reviews

In [64]:
def get_uid(rev): # user id
    user_id=rev['data-userid']
    return None if not user_id else user_id

In [65]:
def get_text(rev): # текст отзыва
    data_id=rev['data-id']
    if not data_id:
        return None
    if data_id==0:
        block=rev.find('p', attrs={'class':'bth__cnt mb10'})
        return None if not block else block.text.strip()
    else:
        data_link="https://tophotels.ru/viewrate/?id="+data_id+"#viewrate"
        full_review=requests.get(data_link, headers={'User-Agent': UserAgent().chrome})# https://tophotels.ru/viewrate/?id=<data-id>#viewrate
        time.sleep(2)
        review_soup = BeautifulSoup(full_review.content, "html.parser")
        block=review_soup.find("p", attrs={'class':'bth__cnt bth__cnt--big'})
        return None if not block else block.text.strip()

In [66]:
def get_mark(rev): # оценка, выставленная в отзыве
    r=rev.find('p', attrs={'class':'bth__cnt mark__cnt'})
    return None if not r else r.text.strip()

In [67]:
def get_datetime(rev): # дата и время, когда был оставлен отзыв, а также интервал времени, прошедший с момента посещения
    info=rev.find('div', attrs={'class':'hint-block'})
    if not info:
        return None, None
    dt=info.find('p', attrs={'class':'bth__cnt'})
    date_time=None if not dt else dt.text.strip()
    delta=info.find('div', attrs={'class':'hint hint--big rate-hint'})
    time_delta=None if not delta else delta.text.strip()
    return date_time, time_delta

In [68]:
def get_rev_line(rev,reference):
    uid=get_uid(rev)
    mark=get_mark(rev)
    date, timedelta=get_datetime(rev)
    text=get_text(rev)
    return {'reference':reference,'user':uid,'mark':mark,'date':date,'time_delta':timedelta,'text':text}

In [69]:
col_names=['reference','user','mark','date','time_delta','text']

In [74]:
data_reviews=pd.DataFrame(columns=col_names)

In [75]:
#data_reviews=pd.read_csv("Crimea_reviews_raw_hotel3.csv")
data_reviews.head()

,reference,user,mark,date,time_delta,text


In [76]:
clear_output()
t1=tqdm(UBK.index[1:],leave=False)
cnt=0
for rec_number in t1:
    n_rev=UBK.at[rec_number,'n_reviews']
    ref=UBK.at[rec_number,'reference']
    first_page=1
    last_page=int(np.ceil(n_rev/25))
    tq1=tqdm(range(first_page,last_page+1), leave=False)
    for i in tq1:
        reviews = get_reviews(page_link(i,ref))
        time.sleep(2)
        tq2=tqdm(reviews, leave=False)
        for rev in tq2:
            data_reviews=data_reviews.append(get_rev_line(rev,ref),ignore_index=True)
            cnt+=1
            if cnt%800==0:
                outp_fname="data_reviews_raw"+str(cnt)+".csv"
                data_reviews.to_csv(outp_fname,header=data_reviews.columns,index=False)
                data_reviews=pd.DataFrame(columns=col_names)
            clear_output()
        tq2.close()
        clear_output()
    tq1.close()
    clear_output()













100%|██████████████████████████████████████████████████████████████████████████████| 321/321 [3:59:05<00:00,  6.73s/it]











                                                                                                                       

## Сохраняем полученные данные в сыром виде

In [77]:
data_reviews.to_csv("UBK_reviews_raw_last.csv",header=data_reviews.columns,index=False)
#data_reviews.head()

In [78]:
len(data_reviews)

619

In [79]:
cnt

4619